In [181]:
import pandas as pd
import numpy as np
import re #usful for searching words in pharagraph
from nltk.corpus import stopwords  #words doesn't add much value to phara (rticals)
from nltk.stem.porter import PorterStemmer #gives a root word
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer #convert text in to feature vectors
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [182]:
df=pd.read_excel('dataset1.xlsx')

In [183]:
df.head()

,dc_indicator,gl,vendor,item_text,gl_acc_name,gl_acc_description,document_reference,label
0,H,61004060,,Export AF - Correction for AI-DN-20210615,Freight inwards charges - Air,This account is used to record the freight cha...,AF PAYMENT,0
1,H,61004060,,Credit Note,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018288,0
2,H,61004060,,Credit Note,Freight inwards charges - Air,This account is used to record the freight cha...,KATF017939,0
3,H,61004060,,Credit Note,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018434,0
4,H,34001010,304285,Import Charges - AIR,Freight inwards charges - Air,This account is used to record the freight cha...,KATF017842,1


In [184]:
len(df['item_text'].unique())

774

In [185]:
df = df.drop_duplicates(subset=['item_text'], keep='first')

In [186]:
df.shape

(774, 8)

In [187]:
df.reset_index(drop=True, inplace=True)

In [188]:
df

,dc_indicator,gl,vendor,item_text,gl_acc_name,gl_acc_description,document_reference,label
0,H,61004060,,Export AF - Correction for AI-DN-20210615,Freight inwards charges - Air,This account is used to record the freight cha...,AF PAYMENT,0
1,H,61004060,,Credit Note,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018288,0
2,H,34001010,304285,Import Charges - AIR,Freight inwards charges - Air,This account is used to record the freight cha...,KATF017842,1
3,H,34001010,304285,,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-22,0
4,H,34001010,304285,CITI Bank test run,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-01,0
...,...,...,...,...,...,...,...,...
769,H,34005030,,Accrued FW fee for import Jun 21/CP hang nhap,Freight inwards charges - Air,This account is used to record the freight cha...,MACS,1
770,H,34001020,304285,Freight Chgs-Air,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018463,1
771,H,34002010,400608,Freight Charges - Air,Freight inwards charges - Air,This account is used to record the freight cha...,S14021090948,1
772,H,34001020,304285,FREIGHT CHARGES,Freight inwards charges - Air,This account is used to record the freight cha...,KATW008188,1


In [189]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saranga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [190]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [191]:
df.shape

(774, 8)

In [192]:
df.isnull().sum()

dc_indicator          0
gl                    0
vendor                0
item_text             0
gl_acc_name           0
gl_acc_description    0
document_reference    0
label                 0
dtype: int64

In [193]:
df.dtypes

dc_indicator          object
gl                     int64
vendor                object
item_text             object
gl_acc_name           object
gl_acc_description    object
document_reference    object
label                  int64
dtype: object

In [194]:
df["messages"] = df["item_text"].astype(str) + " " + df["gl_acc_name"] + " " + df["gl_acc_description"]

In [195]:
df["messages"]

0      Export AF -  Correction for AI-DN-20210615 Fre...
1      Credit Note Freight inwards charges - Air This...
2      Import Charges - AIR Freight inwards charges -...
3        Freight inwards charges - Air This account i...
4      CITI Bank test run Freight inwards charges - A...
                             ...                        
769    Accrued FW fee for import Jun 21/CP hang nhap ...
770    Freight Chgs-Air Freight inwards charges - Air...
771    Freight Charges - Air Freight inwards charges ...
772    FREIGHT CHARGES Freight inwards charges - Air ...
773    FEIGTH CHARGES- AIR Freight inwards charges - ...
Name: messages, Length: 774, dtype: object

In [196]:
#seperate the data and label
X=df.drop(columns="label",axis=1) #axis=1 "removing a column"
y=df["label"]

In [197]:
X

,dc_indicator,gl,vendor,item_text,gl_acc_name,gl_acc_description,document_reference,messages
0,H,61004060,,Export AF - Correction for AI-DN-20210615,Freight inwards charges - Air,This account is used to record the freight cha...,AF PAYMENT,Export AF - Correction for AI-DN-20210615 Fre...
1,H,61004060,,Credit Note,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018288,Credit Note Freight inwards charges - Air This...
2,H,34001010,304285,Import Charges - AIR,Freight inwards charges - Air,This account is used to record the freight cha...,KATF017842,Import Charges - AIR Freight inwards charges -...
3,H,34001010,304285,,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-22,Freight inwards charges - Air This account i...
4,H,34001010,304285,CITI Bank test run,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-01,CITI Bank test run Freight inwards charges - A...
...,...,...,...,...,...,...,...,...
769,H,34005030,,Accrued FW fee for import Jun 21/CP hang nhap,Freight inwards charges - Air,This account is used to record the freight cha...,MACS,Accrued FW fee for import Jun 21/CP hang nhap ...
770,H,34001020,304285,Freight Chgs-Air,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018463,Freight Chgs-Air Freight inwards charges - Air...
771,H,34002010,400608,Freight Charges - Air,Freight inwards charges - Air,This account is used to record the freight cha...,S14021090948,Freight Charges - Air Freight inwards charges ...
772,H,34001020,304285,FREIGHT CHARGES,Freight inwards charges - Air,This account is used to record the freight cha...,KATW008188,FREIGHT CHARGES Freight inwards charges - Air ...


In [198]:
y

0      0
1      0
2      1
3      0
4      0
      ..
769    1
770    1
771    1
772    1
773    1
Name: label, Length: 774, dtype: int64

In [199]:
port_stem=PorterStemmer()

In [200]:
def stemming(content):
    stemmed_content=re.sub("[^a-zA-z]"," ",content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content= " ".join(stemmed_content)
    return stemmed_content

In [201]:
df["messages"] = df["messages"].apply(stemming)

In [202]:
print(df["messages"])

0      export af correct ai dn freight inward charg a...
1      credit note freight inward charg air account u...
2      import charg air freight inward charg air acco...
3      freight inward charg air account use record fr...
4      citi bank test run freight inward charg air ac...
                             ...                        
769    accru fw fee import jun cp hang nhap freight i...
770    freight chg air freight inward charg air accou...
771    freight charg air freight inward charg air acc...
772    freight charg freight inward charg air account...
773    feigth charg air freight inward charg air acco...
Name: messages, Length: 774, dtype: object


In [203]:
df.head()

,dc_indicator,gl,vendor,item_text,gl_acc_name,gl_acc_description,document_reference,label,messages
0,H,61004060,,Export AF - Correction for AI-DN-20210615,Freight inwards charges - Air,This account is used to record the freight cha...,AF PAYMENT,0,export af correct ai dn freight inward charg a...
1,H,61004060,,Credit Note,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018288,0,credit note freight inward charg air account u...
2,H,34001010,304285,Import Charges - AIR,Freight inwards charges - Air,This account is used to record the freight cha...,KATF017842,1,import charg air freight inward charg air acco...
3,H,34001010,304285,,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-22,0,freight inward charg air account use record fr...
4,H,34001010,304285,CITI Bank test run,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-01,0,citi bank test run freight inward charg air ac...


In [204]:
X=df["messages"].values
y=df["label"].values

In [205]:
X

array(['export af correct ai dn freight inward charg air account use record freight charg incur import raw materi product air',
       'credit note freight inward charg air account use record freight charg incur import raw materi product air',
       'import charg air freight inward charg air account use record freight charg incur import raw materi product air',
       'freight inward charg air account use record freight charg incur import raw materi product air',
       'citi bank test run freight inward charg air account use record freight charg incur import raw materi product air',
       'warehous charg hmsc mattakkuliya freight inward charg air account use record freight charg incur import raw materi product air',
       'freight charg vietnam freight inward charg air account use record freight charg incur import raw materi product air',
       'air freight payment freight inward charg air account use record freight charg incur import raw materi product air',
       'credit note f

In [206]:
y

array([0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,

In [207]:
#converting data to numerical data

vectorizer=TfidfVectorizer(max_features=50 , ngram_range=(1,3))
X=vectorizer.fit_transform(X)

In [208]:
print(X)

  (0, 38)	0.14433756729740646
  (0, 43)	0.14433756729740646
  (0, 28)	0.14433756729740646
  (0, 31)	0.14433756729740646
  (0, 13)	0.14433756729740646
  (0, 16)	0.14433756729740646
  (0, 46)	0.14433756729740646
  (0, 49)	0.14433756729740646
  (0, 2)	0.14433756729740646
  (0, 5)	0.14433756729740646
  (0, 11)	0.14433756729740646
  (0, 34)	0.14433756729740646
  (0, 18)	0.14433756729740646
  (0, 40)	0.14433756729740646
  (0, 37)	0.14433756729740646
  (0, 42)	0.14433756729740646
  (0, 27)	0.14433756729740646
  (0, 30)	0.14433756729740646
  (0, 12)	0.14433756729740646
  (0, 15)	0.14433756729740646
  (0, 45)	0.14433756729740646
  (0, 48)	0.14433756729740646
  (0, 1)	0.14433756729740646
  (0, 4)	0.14433756729740646
  (0, 10)	0.14433756729740646
  :	:
  (773, 37)	0.11774918568387145
  (773, 42)	0.11774918568387145
  (773, 27)	0.11774918568387145
  (773, 30)	0.11774918568387145
  (773, 12)	0.11774918568387145
  (773, 15)	0.11774918568387145
  (773, 45)	0.11774918568387145
  (773, 48)	0.1177491856

In [209]:
X.shape

(774, 50)

In [210]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

In [211]:
log_reg=LogisticRegression()

In [212]:
log_reg.fit(X_train,y_train)

LogisticRegression()

In [213]:
#accuracy on train data

X_train_pred=log_reg.predict(X_train)
acc = accuracy_score(X_train_pred,y_train)

In [214]:
print("train data accuracy : ",acc*100,"%")

train data accuracy :  95.47657512116317 %


In [215]:
#accuracy on test data

X_test_pred=log_reg.predict(X_test)
acc=accuracy_score(X_test_pred,y_test)

In [216]:
print("train data accuracy : ",acc*100,"%")

train data accuracy :  92.90322580645162 %


In [217]:
log_reg.predict(X_test[:30])

array([0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0], dtype=int64)

In [218]:
print(y_test[:30])

[1 0 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0]


In [219]:
#### trying bring up a test file

df_test=pd.read_excel('dataset1_test.xlsx')

In [220]:
df_test.head()

,dc_indicator,gl,vendor,item_text,gl_acc_name,gl_acc_description,document_reference,label
0,H,34002010,402349,FEIGTH CHARGES- AIR,Freight inwards charges - Air,This account is used to record the freight cha...,110377,1
1,H,34002010,402349,FEIGTH CHARGES- AIR,Freight inwards charges - Air,This account is used to record the freight cha...,110672A,1
2,H,34002010,402349,FEIGTH CHARGES- AIR,Freight inwards charges - Air,This account is used to record the freight cha...,110673,1
3,H,34002010,402349,FEIGTH CHARGES- AIR,Freight inwards charges - Air,This account is used to record the freight cha...,110720,1
4,H,34002010,402349,FEIGTH CHARGES- AIR,Freight inwards charges - Air,This account is used to record the freight cha...,110721,1


In [221]:
df_test.shape

(674, 8)

In [222]:
df_test.isnull().sum()

dc_indicator          0
gl                    0
vendor                0
item_text             0
gl_acc_name           0
gl_acc_description    0
document_reference    0
label                 0
dtype: int64

In [223]:
df_test["messages_test"] = df_test["item_text"].astype(str) + " " + df_test["gl_acc_name"] + " " + df_test["gl_acc_description"]

In [224]:
df_test["messages_test"]

0      FEIGTH CHARGES- AIR Freight inwards charges - ...
1      FEIGTH CHARGES- AIR Freight inwards charges - ...
2      FEIGTH CHARGES- AIR Freight inwards charges - ...
3      FEIGTH CHARGES- AIR Freight inwards charges - ...
4      FEIGTH CHARGES- AIR Freight inwards charges - ...
                             ...                        
669    Import Air Freight Freight inwards charges - A...
670    IMPORT AIR Freight inwards charges - Air This ...
671    IMPORT AIR FREIGHT Freight inwards charges - A...
672    IMPORT AIR FREIGHT Freight inwards charges - A...
673    IMPORT AIR Freight inwards charges - Air This ...
Name: messages_test, Length: 674, dtype: object

In [225]:
#seperate the data and label
X_testing =df.drop(columns="label",axis=1) #axis=1 "removing a column"

In [226]:
X_testing

,dc_indicator,gl,vendor,item_text,gl_acc_name,gl_acc_description,document_reference,messages
0,H,61004060,,Export AF - Correction for AI-DN-20210615,Freight inwards charges - Air,This account is used to record the freight cha...,AF PAYMENT,export af correct ai dn freight inward charg a...
1,H,61004060,,Credit Note,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018288,credit note freight inward charg air account u...
2,H,34001010,304285,Import Charges - AIR,Freight inwards charges - Air,This account is used to record the freight cha...,KATF017842,import charg air freight inward charg air acco...
3,H,34001010,304285,,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-22,freight inward charg air account use record fr...
4,H,34001010,304285,CITI Bank test run,Freight inwards charges - Air,This account is used to record the freight cha...,CITI-TEST-01,citi bank test run freight inward charg air ac...
...,...,...,...,...,...,...,...,...
769,H,34005030,,Accrued FW fee for import Jun 21/CP hang nhap,Freight inwards charges - Air,This account is used to record the freight cha...,MACS,accru fw fee import jun cp hang nhap freight i...
770,H,34001020,304285,Freight Chgs-Air,Freight inwards charges - Air,This account is used to record the freight cha...,KATF018463,freight chg air freight inward charg air accou...
771,H,34002010,400608,Freight Charges - Air,Freight inwards charges - Air,This account is used to record the freight cha...,S14021090948,freight charg air freight inward charg air acc...
772,H,34001020,304285,FREIGHT CHARGES,Freight inwards charges - Air,This account is used to record the freight cha...,KATW008188,freight charg freight inward charg air account...


In [227]:
df_test["messages_test"] = df_test["messages_test"].apply(stemming)

In [228]:
print(df_test["messages_test"])

0      feigth charg air freight inward charg air acco...
1      feigth charg air freight inward charg air acco...
2      feigth charg air freight inward charg air acco...
3      feigth charg air freight inward charg air acco...
4      feigth charg air freight inward charg air acco...
                             ...                        
669    import air freight freight inward charg air ac...
670    import air freight inward charg air account us...
671    import air freight freight inward charg air ac...
672    import air freight freight inward charg air ac...
673    import air freight inward charg air account us...
Name: messages_test, Length: 674, dtype: object


In [229]:
X_testing=df_test["messages_test"].values
y_testing=df_test["label"].values

In [230]:
#converting data to numerical data

vectorizer=TfidfVectorizer(max_features=50 , ngram_range=(1,3))
X_testing=vectorizer.fit_transform(X_testing)

In [231]:
print(X_testing)

  (0, 38)	0.11401878697385624
  (0, 43)	0.11401878697385624
  (0, 29)	0.11401878697385624
  (0, 32)	0.11401878697385624
  (0, 14)	0.11401878697385624
  (0, 20)	0.11401878697385624
  (0, 46)	0.11401878697385624
  (0, 49)	0.11401878697385624
  (0, 2)	0.11401878697385624
  (0, 5)	0.11401878697385624
  (0, 11)	0.11401878697385624
  (0, 35)	0.11401878697385624
  (0, 24)	0.11401878697385624
  (0, 8)	0.18287656206768774
  (0, 12)	0.1959589514976437
  (0, 17)	0.19841106292934876
  (0, 40)	0.11401878697385624
  (0, 37)	0.11401878697385624
  (0, 42)	0.11401878697385624
  (0, 28)	0.11401878697385624
  (0, 31)	0.11401878697385624
  (0, 13)	0.11401878697385624
  (0, 19)	0.11401878697385624
  (0, 45)	0.11401878697385624
  (0, 48)	0.11401878697385624
  :	:
  (673, 42)	0.12094336549695937
  (673, 28)	0.12094336549695937
  (673, 31)	0.12094336549695937
  (673, 13)	0.12094336549695937
  (673, 19)	0.12094336549695937
  (673, 45)	0.12094336549695937
  (673, 48)	0.12094336549695937
  (673, 1)	0.12094336549

In [232]:
X_testing.shape

(674, 50)

In [233]:
#accuracy on external data

X_testing_pred=log_reg.predict(X_testing)
acc = accuracy_score(X_testing_pred,y_testing)

In [234]:
print("train data accuracy : ",acc*100,"%")

train data accuracy :  89.02077151335311 %
